# Plot data from the VIEWS API, PGM level

In [1]:
# TODO: Needs refactoring and incorporating automatic production of cm maps for the same data selection. CM is currently a separate file. 

In [2]:
import requests
import pandas as pd
import json
import os
from pathlib import Path

#import matplotlib.pyplot as plt

# External
import geopandas as gpd
import sqlalchemy as sa

# VIEWS
from viewser.operations import fetch
from views_mapper2.mapper2 import *
from views_mapper2.BBoxWriter import *
from views_mapper2.dictionary_writer import *
from ingester3.config import source_db_path

## Get forecasts

In [3]:
# List the dataset specified in the URL above

model = 'fatalities001'
release = '2022_09_t01'
dataset = f'{model}_{release}'

# Define the URL you want to fetch data from. 

loa = 'pgm'

FORECASTS = f'https://api.viewsforecasting.org/{dataset}/{loa}'
FORECASTS 


'https://api.viewsforecasting.org/fatalities001_2022_09_t01/pgm'

In [4]:
step = 3 # set step to plot

## Fetch forecasts from VIEWS API

In [5]:
# Request function
r = requests.get(FORECASTS)

#The set page size 
PAGE_SIZE=1000

#Define the full request 
req_url=FORECASTS #+'/'+str()+'?'+'pagesize='+str(PAGE_SIZE)
    
master_list=[]
r=requests.get(req_url)
page_data=r.json()

list(page_data)

['next_page',
 'prev_page',
 'model_tree',
 'models',
 'row_count',
 'page_count',
 'page_cur',
 'start_date',
 'end_date',
 'data']

In [6]:
# The loop to keep fetching pages until all data is accessed 

master_list+=page_data['data']
    
while page_data['next_page'] != '':
    r=requests.get(page_data['next_page'])
    page_data=r.json()

    master_list+=page_data['data']
        
forecasts_original=pd.DataFrame(master_list)

In [7]:
forecasts_original

,pg_id,month_id,sc_pgm_sb_conf_hist_pgm,sc_pgm_sb_natsoc_lgbm,sc_pgm_sb_broad_hurdle_lgbm,sc_pgm_sb_conflictlong_lgbm,sc_pgm_sb_drought_hurdle_lgbm,sc_pgm_sb_dich_main,sc_pgm_sb_tree_lags_d_1_d_2_hur_36,sc_pgm_sb_main,sc_pgm_sb_baseline_lgbm,sc_pgm_sb_sptime_dist_nu1_10_001_hur_36,sc_pgm_sb_conflictlong_hurdle_lgbm,sc_pgm_sb_broad_lgbm,sc_pgm_sb_drought_lgbm,sc_pgm_sb_natsoc_hurdle_lgbm
0,62356,514,0.0008,0.0005,0.0000,0.0004,0.0000,0.0005,0.0004,0.0001,0.0003,0.0001,0.0000,0.0004,0.0011,0.0000
1,79599,514,0.0008,0.0001,0.0000,0.0004,0.0000,0.0005,0.0005,0.0001,0.0003,0.0001,0.0000,0.0002,0.0002,0.0000
2,79600,514,0.0008,0.0004,0.0000,0.0004,0.0000,0.0005,0.0005,0.0001,0.0003,0.0001,0.0000,0.0006,0.0002,0.0000
3,79601,514,0.0008,0.0002,0.0000,0.0004,0.0000,0.0005,0.0005,0.0001,0.0003,0.0001,0.0000,0.0002,0.0002,0.0000
4,80317,514,0.0008,0.0000,0.0000,0.0007,0.0001,0.0005,0.0005,0.0001,0.0006,0.0001,0.0000,0.0004,0.0009,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471955,190496,549,0.0031,0.0101,0.0004,0.0024,0.0000,0.0006,0.0022,0.0241,0.0016,0.0006,0.0008,0.0009,0.0009,0.0003
471956,190507,549,0.0031,0.0004,0.0002,0.0002,0.0000,0.0006,0.0038,0.0013,-0.0004,0.0013,0.0002,0.0000,0.0023,0.0000
471957,190508,549,0.0031,0.0023,0.0000,0.0002,0.0000,0.0006,0.0044,0.0014,-0.0004,0.0013,0.0000,0.0000,0.0026,0.0069
471958,190510,549,0.0031,0.0004,0.0002,0.0002,0.0000,0.0006,0.0045,0.0007,-0.0004,0.0004,0.0003,0.0000,0.0018,0.0001


In [8]:
print(f"Model is: {model}")  # Check the value of model
print(forecasts_original.columns)     # Check the columns in forecasts

if model == 'fatalities001':
    columns_to_keep = ['pg_id', 'month_id', 'sc_pgm_sb_main']
    if all(col in forecasts_original.columns for col in columns_to_keep):
        forecasts = forecasts_original[columns_to_keep].copy()
        print('Here is the filtered dataframe for the fatalities001 model:')
    else:
        print(f"One or more columns from {columns_to_keep} are not in the DataFrame.")

if model == 'fatalities002':
    columns_to_keep = ['pg_id', 'month_id', 'main_mean', 'main_mean_ln']
    if all(col in forecasts_original.columns for col in columns_to_keep):
        forecasts = forecasts_original[columns_to_keep].copy()
        print('Here is the filtered dataframe for the fatalities002 model:')
    else:
        print(f"One or more columns from {columns_to_keep} are not in the DataFrame.")

# Check the final DataFrame
forecasts


Model is: fatalities001
Index(['pg_id', 'month_id', 'sc_pgm_sb_conf_hist_pgm', 'sc_pgm_sb_natsoc_lgbm',
       'sc_pgm_sb_broad_hurdle_lgbm', 'sc_pgm_sb_conflictlong_lgbm',
       'sc_pgm_sb_drought_hurdle_lgbm', 'sc_pgm_sb_dich_main',
       'sc_pgm_sb_tree_lags_d_1_d_2_hur_36', 'sc_pgm_sb_main',
       'sc_pgm_sb_baseline_lgbm', 'sc_pgm_sb_sptime_dist_nu1_10_001_hur_36',
       'sc_pgm_sb_conflictlong_hurdle_lgbm', 'sc_pgm_sb_broad_lgbm',
       'sc_pgm_sb_drought_lgbm', 'sc_pgm_sb_natsoc_hurdle_lgbm'],
      dtype='object')
Here is the filtered dataframe for the fatalities001 model:


,pg_id,month_id,sc_pgm_sb_main
0,62356,514,0.0001
1,79599,514,0.0001
2,79600,514,0.0001
3,79601,514,0.0001
4,80317,514,0.0001
...,...,...,...
471955,190496,549,0.0241
471956,190507,549,0.0013
471957,190508,549,0.0014
471958,190510,549,0.0007


In [9]:
if 'sc_pgm_sb_main' in forecasts.columns:
    forecasts.rename(columns={'sc_pgm_sb_main':'main_mean_ln'}, inplace=True)
forecasts

,pg_id,month_id,main_mean_ln
0,62356,514,0.0001
1,79599,514,0.0001
2,79600,514,0.0001
3,79601,514,0.0001
4,80317,514,0.0001
...,...,...,...
471955,190496,549,0.0241
471956,190507,549,0.0013
471957,190508,549,0.0014
471958,190510,549,0.0007


In [10]:
forecasts['main_mean_ln2'] = np.log1p(forecasts['main_mean_ln']).astype(float)
forecasts['main_mean'] = np.expm1(forecasts['main_mean_ln'])
print('Added a column for the linear and ln2 versions of main_mean_ln. See here:')
forecasts

Added a column for the linear and ln2 versions of main_mean_ln. See here:


,pg_id,month_id,main_mean_ln,main_mean_ln2,main_mean
0,62356,514,0.0001,0.000100,0.000100
1,79599,514,0.0001,0.000100,0.000100
2,79600,514,0.0001,0.000100,0.000100
3,79601,514,0.0001,0.000100,0.000100
4,80317,514,0.0001,0.000100,0.000100
...,...,...,...,...,...
471955,190496,549,0.0241,0.023814,0.024393
471956,190507,549,0.0013,0.001299,0.001301
471957,190508,549,0.0014,0.001399,0.001401
471958,190510,549,0.0007,0.000700,0.000700


In [11]:
# Generate month to plot based on first month of selected dataset and the step chosen above

t = (forecasts['month_id'].min())-1
month_to_plot = t + step 
month_to_plot



516

## Set output folder

In [12]:
import os
home = os.path.expanduser("~")
desktop = home+'/Desktop/' # (Toggle on to save to desktop) Change to your directory
forecasts.to_csv(desktop+'forecasts_pgm.csv', index=False)

## Prep dataframes for plotting

In [13]:
# Get priogrid geometries
engine = sa.create_engine(source_db_path)
gdf_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as pg_id, in_africa, in_me, geom FROM prod.priogrid", 
    engine, 
    geom_col='geom'
)
gdf_master = gdf_master.to_crs(4326)
gdf_pg = gdf_master.copy()

# Get country geometries to overlay as borders
engine = sa.create_engine(source_db_path) 
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, name, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)

gdf_c = gdf_ci_master.copy()

In [14]:
gdf_pg

,pg_id,in_africa,in_me,geom
0,240318,False,False,"POLYGON ((98.5 76.5, 98.5 77, 99 77, 99 76.5, ..."
1,173423,False,False,"POLYGON ((131 30, 131 30.5, 131.5 30.5, 131.5 ..."
2,234640,False,False,"POLYGON ((139.5 72.5, 139.5 73, 140 73, 140 72..."
3,211904,False,False,"POLYGON ((-68.5 57, -68.5 57.5, -68 57.5, -68 ..."
4,218313,False,False,"POLYGON ((-104 61.5, -104 62, -103.5 62, -103...."
...,...,...,...,...
64813,169611,True,True,"POLYGON ((25 27.5, 25 28, 25.5 28, 25.5 27.5, ..."
64814,153089,False,True,"POLYGON ((44 16, 44 16.5, 44.5 16.5, 44.5 16, ..."
64815,172514,False,True,"POLYGON ((36.5 29.5, 36.5 30, 37 30, 37 29.5, ..."
64816,170344,True,True,"POLYGON ((31.5 28, 31.5 28.5, 32 28.5, 32 28, ..."


In [15]:
data = pd.merge(forecasts, gdf_pg, on="pg_id", how="left")
data

,pg_id,month_id,main_mean_ln,main_mean_ln2,main_mean,in_africa,in_me,geom
0,62356,514,0.0001,0.000100,0.000100,True,False,"POLYGON ((37.5 -47, 37.5 -46.5, 38 -46.5, 38 -..."
1,79599,514,0.0001,0.000100,0.000100,True,False,"POLYGON ((19 -35, 19 -34.5, 19.5 -34.5, 19.5 -..."
2,79600,514,0.0001,0.000100,0.000100,True,False,"POLYGON ((19.5 -35, 19.5 -34.5, 20 -34.5, 20 -..."
3,79601,514,0.0001,0.000100,0.000100,True,False,"POLYGON ((20 -35, 20 -34.5, 20.5 -34.5, 20.5 -..."
4,80317,514,0.0001,0.000100,0.000100,True,False,"POLYGON ((18 -34.5, 18 -34, 18.5 -34, 18.5 -34..."
...,...,...,...,...,...,...,...,...
471955,190496,549,0.0241,0.023814,0.024393,False,True,"POLYGON ((27.5 42, 27.5 42.5, 28 42.5, 28 42, ..."
471956,190507,549,0.0013,0.001299,0.001301,False,True,"POLYGON ((33 42, 33 42.5, 33.5 42.5, 33.5 42, ..."
471957,190508,549,0.0014,0.001399,0.001401,False,True,"POLYGON ((33.5 42, 33.5 42.5, 34 42.5, 34 42, ..."
471958,190510,549,0.0007,0.000700,0.000700,False,True,"POLYGON ((34.5 42, 34.5 42.5, 35 42.5, 35 42, ..."


In [16]:
gdf = gpd.GeoDataFrame(data, geometry="geom")
gdf

,pg_id,month_id,main_mean_ln,main_mean_ln2,main_mean,in_africa,in_me,geom
0,62356,514,0.0001,0.000100,0.000100,True,False,"POLYGON ((37.5 -47, 37.5 -46.5, 38 -46.5, 38 -..."
1,79599,514,0.0001,0.000100,0.000100,True,False,"POLYGON ((19 -35, 19 -34.5, 19.5 -34.5, 19.5 -..."
2,79600,514,0.0001,0.000100,0.000100,True,False,"POLYGON ((19.5 -35, 19.5 -34.5, 20 -34.5, 20 -..."
3,79601,514,0.0001,0.000100,0.000100,True,False,"POLYGON ((20 -35, 20 -34.5, 20.5 -34.5, 20.5 -..."
4,80317,514,0.0001,0.000100,0.000100,True,False,"POLYGON ((18 -34.5, 18 -34, 18.5 -34, 18.5 -34..."
...,...,...,...,...,...,...,...,...
471955,190496,549,0.0241,0.023814,0.024393,False,True,"POLYGON ((27.5 42, 27.5 42.5, 28 42.5, 28 42, ..."
471956,190507,549,0.0013,0.001299,0.001301,False,True,"POLYGON ((33 42, 33 42.5, 33.5 42.5, 33.5 42, ..."
471957,190508,549,0.0014,0.001399,0.001401,False,True,"POLYGON ((33.5 42, 33.5 42.5, 34 42.5, 34 42, ..."
471958,190510,549,0.0007,0.000700,0.000700,False,True,"POLYGON ((34.5 42, 34.5 42.5, 35 42.5, 35 42, ..."


## Plot settings

In [17]:
from views_mapper2.mapper2 import Mapper2

def custom_add_colorbar(
    self,
    cmap,
    vmin,
    vmax,
    location="right",
    size="5%",
    pad=0.1,
    transparency=None,
    labelsize=15,
    tickparams=None,
):
    norm = plt.Normalize(vmin, vmax)
    if isinstance(cmap, str):
        cmap = plt.get_cmap(cmap)
    cmap = color.force_alpha_colormap(cmap=cmap, alpha=transparency)
    scalar_to_rgba = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    divider = make_axes_locatable(self.ax)
    self.cax = divider.append_axes(location, size, pad)
    self.cax.tick_params(labelsize=labelsize)

    tickvalues = (
        list(tickparams.values()) if tickparams is not None else None
    )
    self.cbar = plt.colorbar(
        scalar_to_rgba, cax=self.cax, ticks=tickvalues
    )
    if tickparams is not None:
        self.cbar.set_ticklabels(list(tickparams.keys()))
    return self

# Monkey patch the method
Mapper2.add_colorbar = custom_add_colorbar




In [18]:
if month_to_plot == 546:
    month_in_filepath = '546(June2025)'

if month_to_plot == 528:
    month_in_filepath = '528(Dec2023)'

if month_to_plot == 516:
    month_in_filepath = '516(Dec2022)'

month_in_filepath


'516(Dec2022)'

In [19]:
custom_scale = [0, 3, 10, 30, 100, 300]
custom_1p = log1p_dict(custom_scale)

In [21]:
filtered_gdf = gdf[gdf['month_id'] == month_to_plot]  

region = 'ame'

import itertools

cmap = ['rainbow', 'binary']
linewidth = [0.00, 0.05]

for cmaps, linewidths in itertools.product(cmap, linewidth):
    test_map = Mapper2(
        width=10,   # dimension width
        height=10,  # dimension height
        frame_on=True,
        bbox=bbox_from_cid(f'{region}'), 
    ).add_layer(
        gdf=filtered_gdf,  
        cmap=cmaps,
        transparency=1,
        background=None, 
        map_dictionary=custom_1p, 
        edgecolor="black",  # border color choice
        linewidth=linewidths,  # line size choice
        column="main_mean_ln", 
    )
    
    # Determine the line label based on the linewidth
    if linewidths == 0.00:
        line_label = 'no_pg_border'
    else: 
        line_label = 'thin_pg_border'
    
    # Jim procedure for country border creation using the geometries inherent to views3
    ax = test_map.ax
    gdf_c.plot(ax=ax, edgecolor='black', linewidth=0.1, facecolor='None')

    # Print map status
    print(f'{cmaps} with {line_label} map all done!')

    # Save the map with appropriate filename
    test_map.save(f'{desktop}/map_{dataset}_{loa}_s{step}_for_m{month_in_filepath}_{region}_{cmaps}_{line_label}')


rainbow with no_pg_border map all done!
rainbow with thin_pg_border map all done!
binary with no_pg_border map all done!
binary with thin_pg_border map all done!
